<a href="https://colab.research.google.com/github/data-IA-2022/FAO_Project_Tarik_FengFeng/blob/main/Copie_de_FAO_project_Fengfeng_Tarik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center>Data Analyse au sein de la FAO (Food and Agriculture Organization</h1>

<h2>Contexte du projet</h2>

Au sein du service Data Analytics and Prospective de la FAO, vous êtes en charge d'analyser les données produites par les Nations Unies.

Ces analyses conduisent à des rapports de synthèse à destination des décideurs.

Pour êtes également en charge de la partie Prospective, c'est à dire donner des tendances et des évolutions probables quant à la production agricole, à l'alimentation et la nutrition mondiales.

In [5]:
# Copie des fichiers csv du repertoire github et suppréssions des ancien en cas de mise à jour

!rm -r /content/FAO_Project_Tarik_FengFeng
!git clone https://github.com/data-IA-2022/FAO_Project_Tarik_FengFeng.git

Cloning into 'FAO_Project_Tarik_FengFeng'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 29 (delta 12), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (29/29), done.


ValueError: ignored

# Nouvelle section